In [1]:
# Load necessary modules
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import io

In [2]:
BaseURL = "https://www.trouvetamosquee.fr/category/mosquees-de-france/"

Load base URL

In [3]:
page = requests.get(BaseURL)
soup = BeautifulSoup(page.content, 'html.parser')

Strange nesting of divs and hrefs, so we extract all divs for the region-details block, and then parse the html to get the link to each department.

In [4]:
DeptHtml = soup.find_all('div', {'class': 'region-details'})

In [5]:
urls = []

for i in range(len(DeptHtml)):
    urls.append(re.findall("(?P<url>https?://[^\s]+)", str(DeptHtml[i])))

Flatten into a single list

In [6]:
urlsf = [item for sublist in urls for item in sublist]

Need to loop through each url in the flattened list, extract some data, convert to dataframe, and move on. 

In [12]:
# create an empty df
df = pd.DataFrame()

# run loop
for i in urlsf:
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get mosque names
    NameTags = soup.find_all('h2')
    Names = []

    for z in range(len(NameTags)):
        Names.append(NameTags[z].get_text())
        
    # Get mosque addresses
    AddressTags = soup.find_all('h3', class_="address")
    Addresses = []

    for i in range(len(AddressTags)):
        Addresses.append(AddressTags[i].get_text())

    # Construct df
    TempDF = pd.DataFrame({
        'MosqueName': Names, 
        'MosqueAddress': Addresses
    })
    
    df = pd.concat([df, TempDF], sort = False)

In [13]:
df.shape

(2452, 2)

In [14]:
df.head()

,MosqueName,MosqueAddress
0,Grand Mosquée de Strasbourg,6 Rue Averroès - 67000 STRASBOURG
1,MOSQUEE DITIB YUNUS EMRE,3 rue Thomas Mann - 67200 STRASBOURG
2,Mosquée Yunus Merer (DITIB),38 routes de Rohrwiller - 67240 BISCHWILLER
3,Mosquée de Hautepierre,11 Rue Alexandre Dumas - 67200 STRASBOURG
4,Grande Mosquée de Strasbourg,6 Rue Averroès - 67000 STRASBOURG


Save the dataset for future geocoding.

In [ ]:
df.to_csv('../data/to_scrape/CatholicDirectoryScrape.csv')df.to_csv('../data/raw/Mosques/trouvetamosquee_scrape.csv')